# CarDekho Used Car Sales Dataset

Original Dataset: https://www.kaggle.com/datasets/sukritchatterjee/used-cars-dataset-cardekho

The dataset used in this kernel is a cleaned version of the original dataset from Kaggle. It has 13 columns containing information about the make, model, usage patterns and specifications of over 15,000 used cars, scraped from the CarDekho website.

## Goal

The goal of this notebook is to use the dataset as a resource for learning XGBoost Regression. The model will use independent features like make, model, usage patterns and specifications to predict selling price.

In [91]:
!pip install xgboost

In [92]:
# Importing required modules
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
import torch

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer 
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV

In [93]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [94]:
%matplotlib inline
warnings.filterwarnings('ignore')  # Comment out to see warnings

# Importing the Dataset

In [95]:
df = pd.read_csv("/kaggle/input/cardekho-cleaned/cardekho.csv", index_col=[0])

In [96]:
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15411 entries, 0 to 19543
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   car_name           15411 non-null  object 
 1   brand              15411 non-null  object 
 2   model              15411 non-null  object 
 3   vehicle_age        15411 non-null  int64  
 4   km_driven          15411 non-null  int64  
 5   seller_type        15411 non-null  object 
 6   fuel_type          15411 non-null  object 
 7   transmission_type  15411 non-null  object 
 8   mileage            15411 non-null  float64
 9   engine             15411 non-null  int64  
 10  max_power          15411 non-null  float64
 11  seats              15411 non-null  int64  
 12  selling_price      15411 non-null  int64  
dtypes: float64(2), int64(5), object(6)
memory usage: 1.6+ MB


In [98]:
df.describe()

,vehicle_age,km_driven,mileage,engine,max_power,seats,selling_price
count,15411.000000,1.541100e+04,15411.000000,15411.000000,15411.000000,15411.000000,1.541100e+04
mean,6.036338,5.561648e+04,19.701151,1486.057751,100.588254,5.325482,7.749711e+05
std,3.013291,5.161855e+04,4.171265,521.106696,42.972979,0.807628,8.941284e+05
min,0.000000,1.000000e+02,4.000000,793.000000,38.400000,0.000000,4.000000e+04
25%,4.000000,3.000000e+04,17.000000,1197.000000,74.000000,5.000000,3.850000e+05
50%,6.000000,5.000000e+04,19.670000,1248.000000,88.500000,5.000000,5.560000e+05
75%,8.000000,7.000000e+04,22.700000,1582.000000,117.300000,5.000000,8.250000e+05
max,29.000000,3.800000e+06,33.540000,6592.000000,626.000000,9.000000,3.950000e+07


# Data Cleaning

* Handle Typographical Errors
* Handle Missing Values
* Handle Duplicates
* Check Datatypes

## Handling Typographical Errors

In [99]:
# Printing unique entries in all categorical columns
# Getting a list of all categorical columns
cat_cols = df.select_dtypes('object').columns.to_list()
# Printing unique entries to check for data entry errors
for column in cat_cols:
    print(df[column].value_counts(), end="\n\n")

car_name
Hyundai i20           906
Maruti Swift Dzire    890
Maruti Swift          781
Maruti Alto           778
Honda City            757
                     ... 
Mercedes-AMG C          1
Tata Altroz             1
Ferrari GTC4Lusso       1
Hyundai Aura            1
Force Gurkha            1
Name: count, Length: 121, dtype: int64

brand
Maruti           4992
Hyundai          2982
Honda            1485
Mahindra         1011
Toyota            793
Ford              790
Volkswagen        620
Renault           536
BMW               439
Tata              430
Mercedes-Benz     337
Skoda             334
Audi              192
Datsun            170
Jaguar             59
Land Rover         51
Jeep               41
Kia                32
Porsche            21
Volvo              20
MG                 19
Mini               17
Nissan             11
Lexus              10
Isuzu               8
Bentley             3
Maserati            2
ISUZU               2
Ferrari             1
Mercedes-AMG        1

## Handling Missing Values

In [100]:
# Checking for null values
df.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

## Handling Duplicates

In [101]:
df[df.duplicated(keep=False)]

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
31,Maruti Swift Dzire,Maruti,Swift Dzire,6,90000,Individual,Diesel,Manual,26.59,1248,74.00,5,600000
90,Honda City,Honda,City,8,70000,Individual,Petrol,Manual,16.80,1497,116.30,5,545000
210,Maruti Alto,Maruti,Alto,2,25000,Individual,Petrol,Manual,24.70,796,47.30,5,310000
261,Honda City,Honda,City,8,70000,Individual,Petrol,Manual,16.80,1497,116.30,5,545000
281,Renault KWID,Renault,KWID,4,35000,Individual,Petrol,Automatic,24.04,999,67.00,5,350000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19311,Maruti Swift,Maruti,Swift,8,80000,Individual,Diesel,Manual,22.90,1248,74.00,5,350000
19434,Maruti Wagon R,Maruti,Wagon R,6,50000,Individual,CNG,Manual,26.60,998,58.16,5,450000
19489,Tata Tiago,Tata,Tiago,4,30000,Individual,Petrol,Manual,23.84,1199,84.00,5,350000
19505,Hyundai Grand,Hyundai,Grand,6,30000,Individual,Petrol,Manual,18.90,1197,82.00,5,450000


In [102]:
# Dropping duplicate rows
df = df.drop_duplicates(keep='first', ignore_index=True)

In [103]:
# Final check for duplicates
df[df.duplicated(keep=False)].shape

(0, 13)

## Handling Datatypes

In [104]:
df.dtypes

car_name              object
brand                 object
model                 object
vehicle_age            int64
km_driven              int64
seller_type           object
fuel_type             object
transmission_type     object
mileage              float64
engine                 int64
max_power            float64
seats                  int64
selling_price          int64
dtype: object

In [105]:
cat_cols

['car_name', 'brand', 'model', 'seller_type', 'fuel_type', 'transmission_type']

In [106]:
df[cat_cols] = df[cat_cols].astype('category')
df[cat_cols].dtypes

car_name             category
brand                category
model                category
seller_type          category
fuel_type            category
transmission_type    category
dtype: object

# Train Test Split

In [107]:
# Independent Features
X = df.drop(columns='selling_price', axis=1)
# Dependent Features
y = df['selling_price']

In [108]:
X.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [109]:
y.head()

0    120000
1    550000
2    215000
3    226000
4    570000
Name: selling_price, dtype: int64

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((12195, 12), (3049, 12))

# Features Encoding

In [111]:
# Calculating number of unique classes in cateegorical features
unique_values = df[cat_cols].nunique()
unique_values# Train Test Split

car_name             121
brand                 32
model                120
seller_type            3
fuel_type              5
transmission_type      2
dtype: int64

* OneHotEncoder : seller_type, fuel_type, transmission_type have low number of unique values. They can be OneHotEncoded
* OrdinalEncoder: car_name, brand and model have higher number of unique features. They have to be encoded using OrdinalEncoder

In [112]:
# OneHotEncoder columns
onehot_cols = unique_values[unique_values <= 5].index.to_list()
# Ordinal Encoder Columns
ordinal_cols = unique_values[unique_values > 5].index.to_list()

onehot_cols, ordinal_cols

(['seller_type', 'fuel_type', 'transmission_type'],
 ['car_name', 'brand', 'model'])

In [113]:
# Initializing Column Transformer
pre_processor = ColumnTransformer(transformers=[('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'), onehot_cols),
                                               ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), ordinal_cols)], remainder='passthrough', n_jobs=-1)

In [114]:
# Pre-processing X_train
X_train = pre_processor.fit_transform(X_train)

In [115]:
# Pre-processing X_test
X_test = pre_processor.transform(X_test)

In [116]:
encoded_cols = pre_processor.get_feature_names_out()
encoded_cols

array(['onehot__seller_type_Individual',
       'onehot__seller_type_Trustmark Dealer', 'onehot__fuel_type_Diesel',
       'onehot__fuel_type_Electric', 'onehot__fuel_type_LPG',
       'onehot__fuel_type_Petrol', 'onehot__transmission_type_Manual',
       'ordinal__car_name', 'ordinal__brand', 'ordinal__model',
       'remainder__vehicle_age', 'remainder__km_driven',
       'remainder__mileage', 'remainder__engine', 'remainder__max_power',
       'remainder__seats'], dtype=object)

In [117]:
# # Making X_train and X_test pandas dataframes
# X_train = pd.DataFrame(X_train, columns = encoded_cols)
# X_test = pd.DataFrame(X_test, columns = encoded_cols)
# X_train.head()

In [118]:
# X_train.dtypes

In [119]:
# # Changing data types of ordinal columns to category
# ordinal_columns = X_train.filter(like='ordinal').columns.tolist()
# X_train[ordinal_columns] = X_train[ordinal_columns].astype('category')
# X_test[ordinal_columns] = X_test[ordinal_columns].astype('category')
# X_train.dtypes

In [120]:
type(X_train), type(X_test), type(y_train), type(y_test)

(numpy.ndarray,
 numpy.ndarray,
 pandas.core.series.Series,
 pandas.core.series.Series)

# Sample XGBoost Model

In [121]:
# Initializing
sample_model =XGBRegressor(random_state=42) # random_state for reproducibility

In [122]:
# Fitting to X_train, y_train
sample_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [123]:
# Sample Prediction
sample_y_pred = sample_model.predict(X_test)

## Sample Model Scores

In [124]:
sample_r2 = r2_score(y_test, sample_y_pred)
sample_rmse = np.sqrt(mean_squared_error(y_test, sample_y_pred))
sample_mae = mean_absolute_error(y_test, sample_y_pred)

In [125]:
print("R2 score of sample model:", sample_r2)
print("RMSE of sample model:", sample_rmse)
print("MAE score of sample model:", sample_mae)

R2 score of sample model: 0.9302055655044705
RMSE of sample model: 206001.1396357199
MAE score of sample model: 95539.9842020437


In [126]:
# Train Score and Test Score Comparison
print(f"Train Score: {sample_model.score(X_train, y_train):.2f}\nTest Score: {sample_r2:.2f}")

Train Score: 0.99
Test Score: 0.93


# Hyperparameter Tuning

In [127]:
### Initializing Hyperparameters
param_distributions = {"learning_rate": [0.1, 0.01],
                  "max_depth": [5, 8, 12, 20, 30],
                  "n_estimators": [100, 200, 300],
                  "colsample_bytree": [0.5, 0.8, 1, 0.3, 0.4]}

In [128]:
# Initializing Tuner
rcv = RandomizedSearchCV(estimator=XGBRegressor(), param_distributions=param_distributions, 
                         scoring='r2',
                         n_jobs=-1, n_iter=50, cv=2)

In [129]:
# Finding best model
rcv.fit(X_train, y_train)

RandomizedSearchCV(cv=2,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          max_delta_step=None, max_depth=None,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=None, ...),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.5, 0.8, 1, 0.3,
                                                             0.4],
                                        'learning_rate': [0.1, 0.01],
                                        'max_depth': [5, 8, 12, 20, 30],
                                        'n_estimators': [100, 200, 300]},
                   scoring='r2')

In [130]:
model = rcv.best_estimator_
params = rcv.best_params_
params # Best Parameters

{'n_estimators': 300,
 'max_depth': 8,
 'learning_rate': 0.1,
 'colsample_bytree': 0.4}

In [131]:
model.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 0.4,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.1,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 8,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 300,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

## Model Scores

In [132]:
y_pred = model.predict(X_test)

In [133]:
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

In [134]:
print("R2 score of model:", r2)
print("RMSE of model:", rmse)
print("MAE score of model:", mae)

R2 score of model: 0.9418591592907919
RMSE of model: 188018.19550671833
MAE score of model: 92416.18982145785


In [135]:
# Train Score and Test Score Comparison
print(f"Train Score: {model.score(X_train, y_train):.2f}\nTest Score: {r2:.2f}")

Train Score: 0.99
Test Score: 0.94


In [136]:
# Test Score comparison between sample_model and model after hyperparameter tuning
print("Sample Model R2 Score:", sample_r2, end="\n\n")
print("Model R2 Score after hyperparameter tuning:", r2)

Sample Model R2 Score: 0.9302055655044705

Model R2 Score after hyperparameter tuning: 0.9418591592907919


In [137]:
print(f"There is a change in the R2 score from {sample_r2:.2f} to {r2:.2f}")

There is a change in the R2 score from 0.93 to 0.94


# Saving the model

In [138]:
# Finding best model
best_model = model if r2 > sample_r2 else sample_model
best_model.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 0.4,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.1,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 8,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 300,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [139]:
# Directory where model has to be stored
directory = '/kaggle/working/'
# Getting Current Working Directory
path = os.path.join(os.getcwd(), directory, 'XGBRegressor.json')

model.save_model(path)

In [140]:
print("Model has been pickled at", path)

Model has been pickled at /kaggle/working/XGBRegressor.json
